In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

train['is_train'] = True
test['is_train'] = False

combine = train.append(test).reset_index(drop = True)

combine.head()

In [ ]:
combine['Cabin_Letter'] = combine['Cabin'].str[0].fillna('unknown')
combine['Cabin_Number'] = combine['Cabin'].str[1:].fillna(-1).astype('int')
combine['Embarked'] = combine['Embarked'].fillna('unknown')
combine['Sex'] = combine['Sex'].fillna('unknown')
combine['Parch'] = combine['Parch'].astype('str')
combine['Fare'] = combine['Fare'].fillna(combine['Fare'].mean())
combine['Age'] = combine['Age'].fillna(combine['Age'].mean())
combine['SibSp'] = combine['SibSp'].astype('str')
combine['Pclass'] = combine['Pclass'].astype('str')

# split Ticket in two parts
combine[['ticket_pt2','ticket_pt1']] = combine['Ticket'].str[::-1].str.split(' ', expand=True)
combine['ticket_pt1'] = combine['ticket_pt1'].str[::-1]
combine['ticket_pt2'] = combine['ticket_pt2'].str[::-1]

In [ ]:
# select features for model

target = ['Survived']
features = ['Sex', 'Embarked','Cabin_Letter',  
            'Parch', 'Age', 'SibSp', 'Pclass', 'ticket_pt1', 
            'Cabin_Number', 'Fare']

combine = combine[features + target +['is_train']]

## 
combine = pd.get_dummies(combine)

In [ ]:
combine.head()

In [ ]:
combine['AgeBin']=pd.cut(combine['Age'],[-np.inf, 2, 4, 9, 14, 19, 31, 51, np.inf], right=False,
                         labels = [1, 2, 3, 4, 5, 
                                   6, 7, 8])

In [ ]:
combine= combine.drop(["Age"], axis = 1)

In [ ]:
combine.head()

In [ ]:
combine[['AgeBin']] = combine[['AgeBin']].astype('int')

In [ ]:
## split into train and test dataset
tt = combine[combine.is_train]
tte = combine[combine.is_train==False]

In [ ]:
tt.head()

In [ ]:
%%time

N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS)

target = ['Survived']
features = list(set(combine.columns)-set(target))

X = tt[features]
y = tt[target].values

test_preds = np.zeros((tte.shape[0], N_SPLITS))

acc_all = 0

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = GradientBoostingClassifier(random_state=0,).fit(X_train, y_train)
    
    acc = accuracy_score(y_test, model.predict(X_test))
    acc_all += acc / N_SPLITS
    print(f'Accuracy of fold #{fold}: {acc}')
    
    test_preds[:, fold] = model.predict(tte[features])

tte[target] = test_preds.mean(axis=1).round().astype('int')
print(f'\nMean accuracy: {acc_all}')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

submission.loc[:,'Survived'] = tte['Survived'].values

submission.to_csv('submission_gradient_boost.csv', index = False)

submission.head()

Accuracy = 80.52%

Now, let's try other method to make it better.


##### LGBM

In [ ]:
%%time

N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS)

target = ['Survived']
features = list(set(combine.columns)-set(target))

X = tt[features]
y = tt[target].values

test_preds = np.zeros((tte.shape[0], N_SPLITS))

acc_all = 0

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = LGBMClassifier(
        verbose=0,
        metric="Accuracy",
        random_state=42).fit(X_train, y_train)
    
    acc = accuracy_score(y_test, model.predict(X_test))
    acc_all += acc / N_SPLITS
    print(f'Accuracy of fold #{fold}: {acc}')
    
    test_preds[:, fold] = model.predict(tte[features])

tte[target] = test_preds.mean(axis=1).round().astype('int')
print(f'\nMean accuracy: {acc_all}')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

submission.loc[:,'Survived'] = tte['Survived'].values

submission.to_csv('submission_lgbm.csv', index = False)

submission.head()

Let's go to XGBoost: 


In [ ]:
tte.head()

In [ ]:
%%time

N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS)

target = ['Survived']
features = list(set(combine.columns)-set(target))

X = tt[features]
y = tt[target].values

test_preds = np.zeros((tte.shape[0], N_SPLITS))

acc_all = 0

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = XGBClassifier(
        verbose=0,
        eval_metric="logloss",
        random_state=42).fit(X_train, y_train)
    
    acc = accuracy_score(y_test, model.predict(X_test))
    acc_all += acc / N_SPLITS
    print(f'Accuracy of fold #{fold}: {acc}')
    
    test_preds[:, fold] = model.predict(tte[features])

tte[target] = test_preds.mean(axis=1).round().astype('int')
print(f'\nMean accuracy: {acc_all}')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

submission.loc[:,'Survived'] = tte['Survived'].values

submission.to_csv('submission_xgboost.csv', index = False)

submission.head()

Catboost

In [ ]:
%%time

N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS)

target = ['Survived']
features = list(set(combine.columns)-set(target))

X = tt[features]
y = tt[target].values

test_preds = np.zeros((tte.shape[0], N_SPLITS))

acc_all = 0

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = CatBoostClassifier(
        verbose=0,
        eval_metric="Accuracy",
        random_state=42).fit(X_train, y_train)
    
    acc = accuracy_score(y_test, model.predict(X_test))
    acc_all += acc / N_SPLITS
    print(f'Accuracy of fold #{fold}: {acc}')
    
    test_preds[:, fold] = model.predict(tte[features])

tte[target] = test_preds.mean(axis=1).round().astype('int')
print(f'\nMean accuracy: {acc_all}')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

submission.loc[:,'Survived'] = tte['Survived'].values

submission.to_csv('submission_catboost.csv', index = False)

submission.head()